In [16]:
import tensorflow as tf

In [17]:
print(tf.__version__)

2.7.0


In [18]:
import requests
import pandas as pd
import numpy as np

In [19]:
modelo = tf.keras.models.load_model('pesos/rede1.h5')

In [20]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 900)               1620900   
                                                                 
 dense_1 (Dense)             (None, 1)                 901       
                                                                 
Total params: 1,621,801
Trainable params: 1,621,801
Non-trainable params: 0
_________________________________________________________________


In [25]:


url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDiaTicker?diaNum=20220530&ticker=AZUL4"

In [26]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [27]:
df.head()

,campoX,ticker
0,"1.066348, 1.042959, 1.031026, 1.048687, 1.060...",AZUL4


In [28]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [29]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

(1, 1800)

In [30]:
modelo.predict(X)

array([[0.]], dtype=float32)